### Importando as bibliotecas

In [1870]:
import pandas as pd
import mysql.connector
import numpy as np
import importlib
import os

## Coletivo

### Geral

#### Consulta no banco de dados e criação do DataFrame

In [1871]:
# Configurando o banco de dados
config = {
    "host": "127.0.0.1",
    "user": "admin",
    "password": "1234",
    "database": "bet_dados"
}

In [1872]:
conn = mysql.connector.connect(**config)

In [1873]:
query = """
SELECT
	data,
    campeonato,
    pais,
    casa,
    gols_casa,
    gols_fora,
    fora,
    odd_casa,
    odd_empate,
    odd_fora,
CASE
    WHEN gols_casa > gols_fora THEN casa
        WHEN gols_fora > gols_casa THEN fora
        WHEN gols_casa = gols_fora THEN 'Empate'
  END AS vencedor,
  gols_casa + gols_fora AS total_gols,
CASE
    WHEN gols_casa > gols_fora THEN odd_casa
        WHEN gols_fora > gols_casa THEN odd_fora
        WHEN gols_casa = gols_fora THEN 'Empate'
  END AS odd_vencedora
FROM
	partidas
WHERE
	data >= DATE_SUB(DATE_FORMAT(CURDATE(), '%Y-%m-01'), INTERVAL 1 MONTH)
	AND data <  DATE_FORMAT(CURDATE(), '%Y-%m-01');
"""

In [1874]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_19328\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [1875]:
conn.close()

#### Desempenho

In [1876]:
# ======== GARANTE TIPOS NUMÉRICOS ========
for col in ["gols_casa", "gols_fora"]:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

# ======== MAPA clube → país (a partir de casa/fora + pais) ========
def construir_mapa_pais(df: pd.DataFrame) -> pd.Series:
    # Empilha mandantes e visitantes com a coluna 'pais'
    casa_map = df[["casa", "pais"]].rename(columns={"casa": "clube"})
    fora_map = df[["fora", "pais"]].rename(columns={"fora": "clube"})
    mapa = pd.concat([casa_map, fora_map], ignore_index=True)
    mapa["pais"] = mapa["pais"].astype(str)

    # Mode geral (inclui "Mundo")
    mode_all = (
        mapa.groupby("clube")["pais"]
            .agg(lambda s: s.mode().iat[0] if not s.mode().empty else "Desconhecido")
    )

    # Mode preferindo países diferentes de "Mundo"
    mapa_sem_mundo = mapa[mapa["pais"].str.lower() != "mundo"]
    if not mapa_sem_mundo.empty:
        mode_preferencial = (
            mapa_sem_mundo.groupby("clube")["pais"]
                .agg(lambda s: s.mode().iat[0] if not s.mode().empty else None)
        )
        # Atualiza o mapeamento geral com o preferencial quando existir
        mode_all.update(mode_preferencial.dropna())

    return mode_all  # índice = clube, valores = país

# ======== FUNÇÃO AUXILIAR PARA CALCULAR ESTATÍSTICAS ========
def calcular_tabela(df: pd.DataFrame) -> pd.DataFrame:
    # Resultados como mandante
    casa = df[["casa", "gols_casa", "gols_fora", "vencedor"]].copy()
    casa["clube"] = casa["casa"]
    casa["gols_marcados"] = casa["gols_casa"]
    casa["gols_sofridos"] = casa["gols_fora"]
    casa["vitoria"] = (casa["vencedor"] == casa["casa"]).astype(int)
    casa["empate"] = (casa["vencedor"] == "Empate").astype(int)
    casa["derrota"] = ((casa["vencedor"] != casa["casa"]) & (casa["vencedor"] != "Empate")).astype(int)

    # Resultados como visitante
    fora = df[["fora", "gols_casa", "gols_fora", "vencedor"]].copy()
    fora["clube"] = fora["fora"]
    fora["gols_marcados"] = fora["gols_fora"]
    fora["gols_sofridos"] = fora["gols_casa"]
    fora["vitoria"] = (fora["vencedor"] == fora["fora"]).astype(int)
    fora["empate"] = (fora["vencedor"] == "Empate").astype(int)
    fora["derrota"] = ((fora["vencedor"] != fora["fora"]) & (fora["vencedor"] != "Empate")).astype(int)

    # Junta mandante + visitante
    base = pd.concat([casa, fora], ignore_index=True)

    # ======== AGREGA POR clube ========
    tabela = base.groupby("clube", as_index=False).agg(
        jogos=("clube", "count"),
        vitorias=("vitoria", "sum"),
        empates=("empate", "sum"),
        derrotas=("derrota", "sum"),
        gols_marcados=("gols_marcados", "sum"),
        gols_sofridos=("gols_sofridos", "sum")
    )

    # ======== CÁLCULOS DERIVADOS ========
    tabela["pontos"] = tabela["vitorias"] * 3 + tabela["empates"]
    tabela["saldo_gols"] = tabela["gols_marcados"] - tabela["gols_sofridos"]
    tabela["aproveitamento"] = (tabela["pontos"] / (tabela["jogos"] * 3)) * 100

    # ======== ANEXA PAÍS DO CLUBE ========
    pais_por_clube = construir_mapa_pais(df)  # Series índice=clube
    tabela = tabela.merge(
        pais_por_clube.rename("pais"),
        left_on="clube",
        right_index=True,
        how="left"
    )

    # (opcional) reorganiza as colunas para deixar 'pais' logo após 'clube'
    cols = ["clube", "pais", "jogos", "vitorias", "empates", "derrotas",
            "gols_marcados", "gols_sofridos", "saldo_gols", "pontos", "aproveitamento"]
    tabela = tabela[cols]

    # ======== ORDENAÇÃO (critério solicitado) ========
    tabela = tabela.sort_values(
        by=["aproveitamento", "vitorias", "gols_marcados", "saldo_gols", "jogos", "derrotas", "gols_sofridos"],
        ascending=[False, False, False, False, True, True, True]
    ).reset_index(drop=True)

    tabela.index += 1
    return tabela

# ======== EXECUÇÃO ========
tabela_classificacao = calcular_tabela(df)

# ======== EXIBE RESULTADO ========
pd.set_option("display.max_rows", 20)

In [1877]:
tabela_classificacao = tabela_classificacao[['clube', 'pais', 'pontos','jogos', 'vitorias', 'empates', 'derrotas', 'gols_marcados', 'gols_sofridos', 'saldo_gols', 'aproveitamento']]

tabela_classificacao.head()

,clube,pais,pontos,jogos,vitorias,empates,derrotas,gols_marcados,gols_sofridos,saldo_gols,aproveitamento
1,Bayern de Munique,Alemanha,15,5,5,0,0,21,3,18,100.0
2,Barcelona,Espanha,15,5,5,0,0,16,3,13,100.0
3,Always Ready,Bolívia,15,5,5,0,0,14,2,12,100.0
4,2 de Mayo,Paraguai,15,5,5,0,0,9,3,6,100.0
5,Los Angeles,EUA,12,4,4,0,0,15,4,11,100.0


#### Melhor ataque

In [1878]:
melhor_ataque = (
    tabela_classificacao
    .set_index(["clube", "pais"])["gols_marcados"]
    .sort_values(ascending=False)
    .reset_index()
)

In [1879]:
melhor_ataque = melhor_ataque.head(10)

melhor_ataque

,clube,pais,gols_marcados
0,Bayern de Munique,Alemanha,21
1,Atlético Madrid,Espanha,18
2,Real Madrid,Espanha,17
3,Eintracht Frankfurt,Alemanha,16
4,Barcelona,Espanha,16
5,Bodo/Glimt,Noruega,15
6,Los Angeles,EUA,15
7,Toluca,México,15
8,Independiente/BOL,Bolívia,15
9,Guabirá,Bolívia,15


#### Pior defesa

In [1880]:
pior_defesa = (
    tabela_classificacao
    .set_index(["clube", "pais"])["gols_sofridos"]
    .sort_values(ascending=False)
    .reset_index()
)

In [1881]:
pior_defesa = pior_defesa.head(10)

pior_defesa

,clube,pais,gols_sofridos
0,Totora Real Oruro,Bolívia,18
1,Eintracht Frankfurt,Alemanha,17
2,Shonan Bellmare,Japão,15
3,Moldávia,Europa,15
4,Alianza Universidad,Peru,13
5,Yokohama Marinos,Japão,13
6,Nacional Potosí,Bolívia,13
7,Sportivo Luqueño,Paraguai,13
8,Sandefjord,Noruega,13
9,El Nacional,Equador,12


#### Total de Gols por Campeonato

In [1882]:
def gols_jogos_e_media_por_campeonato(df: pd.DataFrame) -> pd.DataFrame:

    # Garante que a coluna total_gols é numérica (evita sustos com strings)
    if not pd.api.types.is_numeric_dtype(df.get("total_gols", pd.Series(dtype=float))):
        df = df.copy()
        df["total_gols"] = pd.to_numeric(df["total_gols"], errors="coerce").fillna(0)

    out = (
        df.groupby(["campeonato", "pais"])
          .agg(
              total_gols=("total_gols", "sum"),
              jogos=("total_gols", "size"),
          )
          .reset_index()
    )

    # Média de gols por jogo (protege contra divisão por zero)
    out["media_gols"] = (out["total_gols"] / out["jogos"]).fillna(0)

    # (Opcional) arredonda a média para 2 casas
    out["media_gols"] = out["media_gols"].round(2)

    # Ordenação sugerida
    out = (
        out.sort_values(["media_gols", "total_gols"], ascending=[False, False])
           .reset_index(drop=True)
    )

    return out


if __name__ == "__main__":

    resultado = gols_jogos_e_media_por_campeonato(df)

In [1883]:
gols_campeonato = resultado[["campeonato", "pais", "jogos", "total_gols"]]

gols_campeonato = gols_campeonato.sort_values("total_gols", ascending=False)

gols_campeonato = gols_campeonato.head(10)

gols_campeonato

,campeonato,pais,jogos,total_gols
19,Copa da Turquia,Turquia,83,256
9,MLS,EUA,54,191
7,Copa de Portugal,Portugal,51,184
13,Copa da Holanda,Holanda,52,174
11,Eliminatórias,Europa,48,164
4,Copa da Dinamarca,Dinamarca,40,149
8,Boliviano,Bolívia,36,128
52,Brasileiro - Série B,Brasil,53,126
49,Eliminatórias,África,51,124
26,Mexicano,México,38,112


#### Maior Média de Gols por Campeonato

Regra: a liga precisa ter pelo menos 10 jogos.

In [1884]:
maior_media_gols = resultado[resultado["jogos"] >= 10]

maior_media_gols = maior_media_gols.sort_values("media_gols", ascending=False)

maior_media_gols = maior_media_gols.head(10)

maior_media_gols

,campeonato,pais,total_gols,jogos,media_gols
2,Copa da Noruega,Noruega,61,15,4.07
4,Copa da Dinamarca,Dinamarca,149,40,3.72
5,Liga dos Campeões,Europa,100,27,3.70
7,Copa de Portugal,Portugal,184,51,3.61
8,Boliviano,Bolívia,128,36,3.56
9,MLS,EUA,191,54,3.54
10,Bundesliga,Alemanha,95,27,3.52
11,Eliminatórias,Europa,164,48,3.42
12,Eredivisie,Holanda,98,29,3.38
13,Copa da Holanda,Holanda,174,52,3.35


#### Odds com mais vitórias

In [1885]:
def faixas_fixas_odds(df: pd.DataFrame) -> pd.DataFrame:

    df = df.copy()

    # Garante numérico e filtra valores válidos
    for c in ["odd_casa", "odd_fora"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    # ---- Ranges fixos (intervalos fechados à direita) ----
    edges  = [1.01, 1.50, 2.00, 2.50, 3.00, 5.00, 10.00, 20.00, 30.00, 40.00, 50.00, float("inf")]
    labels = [
        "1.01 – 1.50",
        "1.51 – 2.00",
        "2.01 – 2.50",
        "2.51 – 3.00",
        "3.01 – 5.00",
        "5.01 – 10.00",
        "10.01 – 20.00",
        "20.01 – 30.00",
        "30.01 – 40.00",
        "40.01 – 50.00",
        "50.01+",
    ]

    def faixa(odds: pd.Series) -> pd.Series:
        return pd.cut(
            odds,
            bins=edges,
            labels=labels,
            right=True,          # (a, b]
            include_lowest=True  # inclui 1.01
        )

    registros = []

    # Casa
    m_casa = df["odd_casa"].notna() & (df["odd_casa"] >= 1.01)
    if m_casa.any():
        faixas_casa = faixa(df.loc[m_casa, "odd_casa"])
        venceu_casa = (df.loc[m_casa, "vencedor"] == df.loc[m_casa, "casa"]).astype(int)
        # empate conta como derrota -> derrota = 1 - venceu
        registros.append(pd.DataFrame({
            "lado": "Casa",
            "faixa_odd": faixas_casa,
            "vitoria": venceu_casa.values,
            "derrota": (1 - venceu_casa).values
        }))

    # Fora
    m_fora = df["odd_fora"].notna() & (df["odd_fora"] >= 1.01)
    if m_fora.any():
        faixas_fora = faixa(df.loc[m_fora, "odd_fora"])
        venceu_fora = (df.loc[m_fora, "vencedor"] == df.loc[m_fora, "fora"]).astype(int)
        registros.append(pd.DataFrame({
            "lado": "Fora",
            "faixa_odd": faixas_fora,
            "vitoria": venceu_fora.values,
            "derrota": (1 - venceu_fora).values
        }))

    # Empilha casa+fora
    odds_df = pd.concat(registros, ignore_index=True)

    # Remove quaisquer odds fora dos ranges (ex.: <1.01)
    odds_df = odds_df[odds_df["faixa_odd"].notna()].copy()

    # Ordenação na ordem dos labels
    odds_df["faixa_odd"] = odds_df["faixa_odd"].astype(pd.api.types.CategoricalDtype(categories=labels, ordered=True))

    resumo = (
        odds_df.groupby("faixa_odd", observed=True)
               .agg(
                   total_jogos=("lado", "count"),
                   vitorias=("vitoria", "sum"),
                   derrotas=("derrota", "sum"),
               )
               .reset_index()
               .sort_values("faixa_odd")
               .reset_index(drop=True)
    )

    resumo["taxa_vitoria_%"] = (resumo["vitorias"] / resumo["total_jogos"] * 100).round(2)
    return resumo

In [1886]:
resultado = faixas_fixas_odds(df)

resultado

,faixa_odd,total_jogos,vitorias,derrotas,taxa_vitoria_%
0,1.01 – 1.50,361,275,86,76.18
1,1.51 – 2.00,509,268,241,52.65
2,2.01 – 2.50,427,163,264,38.17
3,2.51 – 3.00,341,113,228,33.14
4,3.01 – 5.00,671,161,510,23.99
5,5.01 – 10.00,337,36,301,10.68
6,10.01 – 20.00,90,4,86,4.44
7,20.01 – 30.00,31,0,31,0.00
8,30.01 – 40.00,4,0,4,0.00
9,40.01 – 50.00,4,0,4,0.00


### Primeiro Tempo

#### Carregando a consulta

In [1887]:
conn = mysql.connector.connect(**config)

In [1888]:
query = """
SELECT
    p.data,
    t1.temporada,
    t1.clube,
    p.campeonato,
    p.pais,
    CASE
        WHEN t1.clube = p.casa THEN 'Casa'
        WHEN t1.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    t1.ball_possession,
    t1.expected_goals,
    t1.big_chances,
    t1.total_shots,
    t1.shots_on_target,
    t1.shots_inside_box,
    t1.hit_woodwork,
    t1.corner_kicks,
    t1.passes,
    t1.big_chances_scored,
    t1.accurate_passes,
    t1.through_balls,
    t1.final_third_entries,
    t1.fouled_in_final_third,
    t1.dispossessed,
    t1.duels,
    t1.yellow_cards,
    t1.red_cards
FROM
    1_tempo t1
    INNER JOIN partidas p ON t1.id_jogo = p.id_jogo
WHERE
    p.data >= DATE_SUB(DATE_FORMAT(CURDATE(), '%Y-%m-01'), INTERVAL 1 MONTH)
	AND p.data <  DATE_FORMAT(CURDATE(), '%Y-%m-01');
"""

In [1889]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_19328\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [1890]:
conn.close()

In [1891]:
df = df[(df["campeonato"] != "Eliminatórias") & (df["campeonato"] != "Amistoso")]

#### Posse de Bola

Regra: o clube precisa ter realizado no mínimo 3 jogos no mês.

In [1892]:
df["ball_possession"] = (
    df["ball_possession"]
    .astype(str)
    .str.replace("%", "", regex=False)
    .str.replace(",", ".", regex=False)
    .str.strip()
)

In [1893]:
# Garante tipo numérico (se já estiver ok, mantém)
df["ball_possession"] = pd.to_numeric(df["ball_possession"], errors="coerce")

# Adiciona a contagem de jogos por clube e filtra quem tem > 2
df_filtrado = (
    df.assign(jogos=df.groupby("clube")["clube"].transform("size"))
      .query("jogos > 2")
)

# Calcula a posse de bola apenas para clubes com mais de 2 jogos
posse_bola = (
    df_filtrado
    .groupby("clube", as_index=False)["ball_possession"]
    .mean()
    .rename(columns={"ball_possession": "posse_bola"})
    .sort_values("posse_bola", ascending=False)
    .reset_index(drop=True)
)

In [1894]:
mais_posse_bola = posse_bola.head(10)

mais_posse_bola["posse_bola"] = mais_posse_bola["posse_bola"].round(2)

mais_posse_bola

C:\Users\stefa\AppData\Local\Temp\ipykernel_19328\2865080304.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mais_posse_bola["posse_bola"] = mais_posse_bola["posse_bola"].round(2)


,clube,posse_bola
0,Barcelona,75.00
1,Cerro Porteño,71.25
2,PSG,69.25
3,San Diego,68.67
4,Argentinos Juniors,68.25
5,Bayern de Munique,67.40
6,Mjällby,66.33
7,Shelbourne,66.25
8,CSKA Moscow,65.67
9,Arsenal,64.80


#### xG Médio

In [1895]:
xg_medio = (
    df
    .groupby("clube")["expected_goals"]
    .mean()
    .reset_index(name="xg_medio")
    .sort_values("xg_medio", ascending=False)
    .reset_index(drop=True)
)

In [1896]:
xg_medio["xg_medio"] = xg_medio["xg_medio"].round(2)

xg_medio = xg_medio.head(10)

xg_medio

,clube,xg_medio
0,Sporting,1.54
1,RB Leipzig,1.52
2,Real Madrid,1.46
3,Ludogorets,1.42
4,Brest,1.36
5,Young Boys,1.35
6,Bayern de Munique,1.27
7,San Diego,1.20
8,Excelsior,1.19
9,PSV,1.19


#### Finalizações

In [1897]:
finalizacoes = (
    df
    .groupby("clube")["total_shots"]
    .sum()
    .reset_index(name="finalizacoes")
    .sort_values("finalizacoes", ascending=False)
    .reset_index(drop=True)
)

In [1898]:
finalizacoes = finalizacoes.head(10)

finalizacoes

,clube,finalizacoes
0,Real Madrid,65
1,Al-Hilal,58
2,Novorizontino,54
3,Bayern de Munique,51
4,Once Caldas,50
5,Club Brugge,50
6,Barcelona,48
7,Nacional/PAR,48
8,CRB,47
9,Independiente Santa Fe,46


#### Finalizações Certas

In [1899]:
finalizacoes_certas = (
    df
    .groupby("clube")["shots_on_target"]
    .sum()
    .reset_index(name="finalizacoes_certas")
    .sort_values("finalizacoes_certas", ascending=False)
    .reset_index(drop=True)
)

In [1900]:
finalizacoes_certas = finalizacoes_certas.head(10)

finalizacoes_certas

,clube,finalizacoes_certas
0,Real Madrid,29
1,Bayern de Munique,28
2,Al-Hilal,25
3,Nacional/PAR,21
4,Bolívar,20
5,Al-Nassr,20
6,Inter Miami,20
7,Chicago Fire,19
8,Club Brugge,19
9,Aurora,18


#### Finalizações Dentro da Área

In [1901]:
finalizacoes_area = (
    df
    .groupby("clube")["shots_inside_box"]
    .sum()
    .reset_index(name="finalizacoes_area")
    .sort_values("finalizacoes_area", ascending=False)
    .reset_index(drop=True)
)

In [1902]:
finalizacoes_area = finalizacoes_area.head(10)

finalizacoes_area

,clube,finalizacoes_area
0,Real Madrid,48
1,Al-Hilal,42
2,Club Brugge,37
3,Liverpool,34
4,Nacional/PAR,30
5,Bayern de Munique,30
6,Tottenham,30
7,Botafogo,28
8,Oriente Petrolero,28
9,Athletic Bilbao,28


#### Bolas na Trave

In [1903]:
bola_trave = (
    df
    .groupby("clube")["hit_woodwork"]
    .sum()
    .reset_index(name="bola_trave")
    .sort_values("bola_trave", ascending=False)
    .reset_index(drop=True)
)

In [1904]:
bola_trave = bola_trave.head(10)

bola_trave

,clube,bola_trave
0,Oriente Petrolero,6
1,Milan,5
2,Real Madrid,4
3,Al-Hilal,4
4,Novorizontino,4
5,Mazatlán,3
6,Cuiabá,3
7,Machida Zelvia,3
8,Deportivo Táchira,3
9,Alianza Atlético,3


#### Grandes Chances

In [1905]:
grandes_chances = (
    df
    .groupby("clube")["big_chances"]
    .sum()
    .reset_index(name="grandes_chances")
    .sort_values("grandes_chances", ascending=False)
    .reset_index(drop=True)
)

In [1906]:
grandes_chances = grandes_chances.head(10)

grandes_chances

,clube,grandes_chances
0,Atlético Madrid,15
1,Real Madrid,15
2,Al-Hilal,14
3,Al-Nassr,13
4,Bayern de Munique,12
5,Internazionale,11
6,Milan,11
7,Go Ahead Eagles,10
8,Galatasaray,10
9,Nottingham Forest,10


#### Grandes Chances Aproveitadas

In [1907]:
grandes_chances_aproveitadas = (
    df
    .groupby("clube")["big_chances_scored"]
    .sum()
    .reset_index(name="grandes_chances_aproveitadas")
    .sort_values("grandes_chances_aproveitadas", ascending=False)
    .reset_index(drop=True)
)

In [1908]:
grandes_chances_aproveitadas = grandes_chances_aproveitadas.head(10)

grandes_chances_aproveitadas

,clube,grandes_chances_aproveitadas
0,Bayern de Munique,7
1,Atlético Madrid,7
2,Palmeiras,7
3,Al-Hilal,6
4,Los Angeles,6
5,Real Madrid,6
6,Columbus Crew,6
7,Cienciano,5
8,Juárez,5
9,Al-Nassr,5


#### Escanteios

In [1909]:
escanteios = (
    df
    .groupby("clube")["corner_kicks"]
    .sum()
    .reset_index(name="escanteios")
    .sort_values("escanteios", ascending=False)
    .reset_index(drop=True)
)

In [1910]:
escanteios = escanteios.head(10)

escanteios

,clube,escanteios
0,Tottenham,31
1,Atlético Madrid,24
2,Feyenoord,22
3,Novorizontino,22
4,Barcelona,22
5,Al-Ahli,21
6,América de Cali,21
7,Fenerbahçe,21
8,Real Madrid,20
9,Viking,20


#### Passes

In [1911]:
passes = (
    df
    .groupby("clube")["passes"]
    .sum()
    .reset_index(name="passes")
    .sort_values("passes", ascending=False)
    .reset_index(drop=True)
)

In [1912]:
passes = passes.head(10)

passes

,clube,passes
0,Liverpool,1811
1,Real Madrid,1788
2,Barcelona,1779
3,Club Brugge,1768
4,Atlético Madrid,1744
5,Fluminense,1707
6,Bayern de Munique,1677
7,Inter Miami,1672
8,Once Caldas,1658
9,Chelsea,1642


#### Passes Certos

In [1913]:
passes_certos = (
    df
    .groupby("clube")["accurate_passes"]
    .sum()
    .reset_index(name="passes_certos")
    .sort_values("passes_certos", ascending=False)
    .reset_index(drop=True)
)

In [1914]:
passes_certos = passes_certos.head(10)

passes_certos

,clube,passes_certos
0,Barcelona,1602
1,Liverpool,1601
2,Real Madrid,1590
3,Atlético Madrid,1547
4,Club Brugge,1529
5,Bayern de Munique,1508
6,Inter Miami,1490
7,Fluminense,1487
8,Chelsea,1453
9,Al-Hilal,1406


#### Passes em Profundidade

In [1915]:
passes_profundidade = (
    df
    .groupby("clube")["through_balls"]
    .sum()
    .reset_index(name="passes_profundidade")
    .sort_values("passes_profundidade", ascending=False)
    .reset_index(drop=True)
)

In [1916]:
passes_profundidade = passes_profundidade.head(10)

passes_profundidade

,clube,passes_profundidade
0,PSG,6
1,Liverpool,6
2,Genoa,6
3,Milan,6
4,Bayern de Munique,6
5,Inter Miami,6
6,Feyenoord,5
7,Chelsea,5
8,Al-Ittihad,5
9,Club Brugge,5


#### Entradas no Terço Final

In [1917]:
entradas_terco_final = (
    df
    .groupby("clube")["final_third_entries"]
    .sum()
    .reset_index(name="entradas_terco_final")
    .sort_values("entradas_terco_final", ascending=False)
    .reset_index(drop=True)
)

In [1918]:
entradas_terco_final = entradas_terco_final.head(10)

entradas_terco_final

,clube,entradas_terco_final
0,Independiente Del Valle,243
1,Once Caldas,238
2,Real Madrid,234
3,Barcelona,213
4,Sanfrecce Hiroshima,213
5,Al-Hilal,205
6,Liverpool,198
7,Nacional Potosí,195
8,Novorizontino,193
9,Always Ready,191


#### Faltas Sofridas no Terço Final

In [1919]:
faltas_terco_final = (
    df
    .groupby("clube")["fouled_in_final_third"]
    .sum()
    .reset_index(name="faltas_terco_final")
    .sort_values("faltas_terco_final", ascending=False)
    .reset_index(drop=True)
)

In [1920]:
faltas_terco_final = faltas_terco_final.head(10)

faltas_terco_final

,clube,faltas_terco_final
0,Athletico-PR,12
1,Atlético-GO,12
2,Amazonas,11
3,Al-Hilal,11
4,Fredrikstad,10
5,Portuguesa/VEN,10
6,Chelsea,10
7,Akhmat Grozny,10
8,Atlas,9
9,Athletic Bilbao,9


#### Perda da Posse de Bola

In [1921]:
perda_posse = (
    df
    .groupby("clube")["dispossessed"]
    .sum()
    .reset_index(name="perda_posse")
    .sort_values("perda_posse", ascending=False)
    .reset_index(drop=True)
)

In [1922]:
perda_posse = perda_posse.head(10)

perda_posse

,clube,perda_posse
0,Aston Villa,40
1,Lanús,33
2,Remo,33
3,Yaracuyanos,32
4,Benfica,32
5,Athletic Bilbao,30
6,Vasco,28
7,Vélez Sarsfield,28
8,Botafogo,28
9,Atlético,28


#### Duelos Ganhos

In [1923]:
df["duels"] = (
    df["duels"]
    .astype(str)
    .str.replace("%", "", regex=False)
    .str.replace(",", ".", regex=False)
    .str.strip()
)

In [1924]:
df["duels"] = pd.to_numeric(df["duels"], errors="coerce")

In [1925]:
# Conta jogos por clube e filtra quem tem mais de 2
df_filtrado = (
    df.assign(jogos=df.groupby("clube")["clube"].transform("size"))
      .query("jogos > 2")
)

# Calcula a média de duelos ganhos apenas para os clubes com > 2 jogos
duelos_ganhos = (
    df_filtrado
    .groupby("clube", as_index=False)["duels"]
    .mean()
    .rename(columns={"duels": "duelos_ganhos"})
    .sort_values("duelos_ganhos", ascending=False)
    .reset_index(drop=True)
)

In [1926]:
duelos_ganhos = duelos_ganhos.head(10)

duelos_ganhos["duelos_ganhos"] = duelos_ganhos["duelos_ganhos"].astype(int)

duelos_ganhos

,clube,duelos_ganhos
0,Nagoya Grampus,63
1,AEK,63
2,Águilas Doradas,59
3,Fulham,59
4,Al-Shabab,58
5,Alverca,58
6,Derry City,58
7,Heracles Almelo,58
8,Paris,58
9,Monagas,58


#### Cartões Amarelos

In [1927]:
amarelos = (
    df
    .groupby("clube")["yellow_cards"]
    .sum()
    .reset_index(name="amarelos")
    .sort_values("amarelos", ascending=False)
    .reset_index(drop=True)
)

In [1928]:
amarelos = amarelos.head(10)

amarelos

,clube,amarelos
0,Vélez Sarsfield,12
1,Deportivo Pasto,12
2,Oriente Petrolero,10
3,Totora Real Oruro,10
4,Avaí,9
5,Vila Nova-GO,9
6,AZ Alkmaar,9
7,Atlético Bucaramanga,9
8,River Plate,8
9,Real Sociedad,8


#### Cartões Vermelhos

In [1929]:
vermelhos_antes = (
    df
    .groupby("clube")["red_cards"]
    .sum()
    .reset_index(name="vermelhos")
    .sort_values("vermelhos", ascending=False)
    .reset_index(drop=True)
)

In [1930]:
vermelhos_1t = vermelhos_antes[vermelhos_antes["vermelhos"] > 1]

vermelhos = vermelhos_1t.head(10)

vermelhos

,clube,vermelhos
0,Vélez Sarsfield,2
1,El Nacional,2
2,Al-Shabab,2
3,Deportivo Táchira,2
4,Cruz Azul,2
5,OFI Crete,2
6,Al-Fateh,2
7,Aurora,2
8,La Equidad,2


### Segundo Tempo

#### Carregando a consulta

In [1931]:
conn = mysql.connector.connect(**config)

In [1932]:
query = """
SELECT
    p.data,
    t2.temporada,
    t2.clube,
    p.campeonato,
    p.pais,
    CASE
        WHEN t2.clube = p.casa THEN 'Casa'
        WHEN t2.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    t2.ball_possession,
    t2.expected_goals,
    t2.big_chances,
    t2.total_shots,
    t2.shots_on_target,
    t2.shots_inside_box,
    t2.hit_woodwork,
    t2.corner_kicks,
    t2.passes,
    t2.big_chances_scored,
    t2.accurate_passes,
    t2.through_balls,
    t2.final_third_entries,
    t2.fouled_in_final_third,
    t2.dispossessed,
    t2.duels,
    t2.yellow_cards,
    t2.red_cards
FROM
    2_tempo t2
    INNER JOIN partidas p ON t2.id_jogo = p.id_jogo
WHERE
    p.data >= DATE_SUB(DATE_FORMAT(CURDATE(), '%Y-%m-01'), INTERVAL 1 MONTH)
	AND p.data <  DATE_FORMAT(CURDATE(), '%Y-%m-01');
"""

In [1933]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_19328\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [1934]:
conn.close()

In [1935]:
df = df[(df["campeonato"] != "Eliminatórias") & (df["campeonato"] != "Amistoso")]

#### Posse de Bola

Regra: o clube precisa ter realizado no mínimo 3 jogos no mês.

In [1936]:
df["ball_possession"] = (
    df["ball_possession"]
    .astype(str)
    .str.replace("%", "", regex=False)
    .str.replace(",", ".", regex=False)
    .str.strip()
)

In [1937]:
# Garante tipo numérico (se já estiver ok, mantém)
df["ball_possession"] = pd.to_numeric(df["ball_possession"], errors="coerce")

# Adiciona a contagem de jogos por clube e filtra quem tem > 2
df_filtrado = (
    df.assign(jogos=df.groupby("clube")["clube"].transform("size"))
      .query("jogos > 2")
)

# Calcula a posse de bola apenas para clubes com mais de 2 jogos
posse_bola = (
    df_filtrado
    .groupby("clube", as_index=False)["ball_possession"]
    .mean()
    .rename(columns={"ball_possession": "posse_bola"})
    .sort_values("posse_bola", ascending=False)
    .reset_index(drop=True)
)

In [1938]:
mais_posse_bola_2t = posse_bola.head(10)

mais_posse_bola_2t["posse_bola"] = mais_posse_bola_2t["posse_bola"].round(2)

mais_posse_bola_2t

C:\Users\stefa\AppData\Local\Temp\ipykernel_19328\1579566151.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mais_posse_bola_2t["posse_bola"] = mais_posse_bola_2t["posse_bola"].round(2)


,clube,posse_bola
0,Cerro Porteño,73.25
1,Barcelona,69.20
2,Fenerbahçe,67.80
3,Liverpool,67.17
4,San Diego,66.33
5,Arsenal,66.20
6,CRB,65.60
7,PSG,65.25
8,Levadiakos,65.00
9,Spartak Moscow,64.67


#### xG Médio

In [1939]:
xg_medio = (
    df
    .groupby("clube")["expected_goals"]
    .mean()
    .reset_index(name="xg_medio")
    .sort_values("xg_medio", ascending=False)
    .reset_index(drop=True)
)

In [1940]:
xg_medio["xg_medio"] = xg_medio["xg_medio"].round(2)

xg_medio_2t = xg_medio.head(10)

xg_medio_2t

,clube,xg_medio
0,Bayern de Munique,1.85
1,Slavia Praha,1.50
2,Ludogorets,1.50
3,PSV,1.49
4,Atlanta United,1.48
5,Sporting,1.44
6,Twente,1.42
7,Celtic,1.39
8,Strasbourg,1.38
9,Union Saint-Gilloise,1.34


#### Finalizações

In [1941]:
finalizacoes = (
    df
    .groupby("clube")["total_shots"]
    .sum()
    .reset_index(name="finalizacoes")
    .sort_values("finalizacoes", ascending=False)
    .reset_index(drop=True)
)

In [1942]:
finalizacoes_2t = finalizacoes.head(10)

finalizacoes_2t

,clube,finalizacoes
0,Chapecoense,59
1,Liverpool,59
2,Al-Hilal,58
3,Bayern de Munique,57
4,Club Brugge,56
5,América de Cali,56
6,Cuiabá,55
7,Independiente/BOL,55
8,San Antonio Bulo-Bulo,55
9,Barcelona,55


#### Finalizações Certas

In [1943]:
finalizacoes_certas = (
    df
    .groupby("clube")["shots_on_target"]
    .sum()
    .reset_index(name="finalizacoes_certas")
    .sort_values("finalizacoes_certas", ascending=False)
    .reset_index(drop=True)
)

In [1944]:
finalizacoes_certas_2t = finalizacoes_certas.head(10)

finalizacoes_certas_2t

,clube,finalizacoes_certas
0,Barcelona,28
1,Bayern de Munique,27
2,Inter Miami,24
3,Independiente/BOL,23
4,Cuiabá,23
5,Atlético Madrid,23
6,Once Caldas,22
7,Real Madrid,21
8,Aurora,21
9,América de Cali,20


#### Finalizações Dentro da Área

In [1945]:
finalizacoes_area = (
    df
    .groupby("clube")["shots_inside_box"]
    .sum()
    .reset_index(name="finalizacoes_area")
    .sort_values("finalizacoes_area", ascending=False)
    .reset_index(drop=True)
)

In [1946]:
finalizacoes_area_2t = finalizacoes_area.head(10)

finalizacoes_area_2t

,clube,finalizacoes_area
0,Club Brugge,41
1,América de Cali,41
2,Bayern de Munique,41
3,Al-Hilal,39
4,Cuiabá,39
5,Internazionale,39
6,Barcelona,39
7,St. Louis City,38
8,Liverpool,38
9,Chapecoense,36


#### Bolas na Trave

In [1947]:
bola_trave = (
    df
    .groupby("clube")["hit_woodwork"]
    .sum()
    .reset_index(name="bola_trave")
    .sort_values("bola_trave", ascending=False)
    .reset_index(drop=True)
)

In [1948]:
bola_trave_2t = bola_trave.head(10)

bola_trave_2t

,clube,bola_trave
0,Bayern de Munique,5
1,Sanfrecce Hiroshima,4
2,Inter Miami,4
3,Nacional Potosí,3
4,Coritiba,3
5,Atlético,3
6,Barcelona,3
7,Gamba Osaka,3
8,Barnsley,3
9,Fenerbahçe,3


#### Grandes Chances

In [1949]:
grandes_chances = (
    df
    .groupby("clube")["big_chances"]
    .sum()
    .reset_index(name="grandes_chances")
    .sort_values("grandes_chances", ascending=False)
    .reset_index(drop=True)
)

In [1950]:
grandes_chances_2t = grandes_chances.head(10)

grandes_chances_2t

,clube,grandes_chances
0,Bayern de Munique,17
1,Internazionale,16
2,Liverpool,14
3,Atlético Madrid,14
4,Inter Miami,14
5,PSV,13
6,Real Madrid,12
7,Go Ahead Eagles,12
8,Shelbourne,12
9,Tijuana,12


#### Grandes Chances Aproveitadas

In [1951]:
grandes_chances_aproveitadas = (
    df
    .groupby("clube")["big_chances_scored"]
    .sum()
    .reset_index(name="grandes_chances_aproveitadas")
    .sort_values("grandes_chances_aproveitadas", ascending=False)
    .reset_index(drop=True)
)

In [1952]:
grandes_chances_aproveitadas_2t = grandes_chances_aproveitadas.head(10)

grandes_chances_aproveitadas_2t

,clube,grandes_chances_aproveitadas
0,Manchester City,7
1,Al-Taawoun,7
2,Barcelona,7
3,Atlético Madrid,6
4,Bayern de Munique,6
5,Nashville,6
6,Real Madrid,6
7,Juventus,5
8,Inter Miami,5
9,Tijuana,5


#### Escanteios

In [1953]:
escanteios = (
    df
    .groupby("clube")["corner_kicks"]
    .sum()
    .reset_index(name="escanteios")
    .sort_values("escanteios", ascending=False)
    .reset_index(drop=True)
)

In [1954]:
escanteios_2t = escanteios.head(10)

escanteios_2t

,clube,escanteios
0,Al-Nassr,25
1,Chapecoense,24
2,Independiente Del Valle,24
3,Liverpool,24
4,América/MEX,23
5,Al-Hilal,22
6,Sanfrecce Hiroshima,22
7,Al-Ahli,22
8,Manta,22
9,São Paulo,21


#### Passes

In [1955]:
passes = (
    df
    .groupby("clube")["passes"]
    .sum()
    .reset_index(name="passes")
    .sort_values("passes", ascending=False)
    .reset_index(drop=True)
)

In [1956]:
passes_2t = passes.head(10)

passes_2t

,clube,passes
0,Liverpool,1826
1,Inter Miami,1738
2,Barcelona,1706
3,Club Brugge,1643
4,Atlético,1605
5,Al-Hilal,1603
6,Bayern de Munique,1602
7,Real Madrid,1530
8,Chelsea,1524
9,Benfica,1523


#### Passes Certos

In [1957]:
passes_certos = (
    df
    .groupby("clube")["accurate_passes"]
    .sum()
    .reset_index(name="passes_certos")
    .sort_values("passes_certos", ascending=False)
    .reset_index(drop=True)
)

In [1958]:
passes_certos_2t = passes_certos.head(10)

passes_certos_2t

,clube,passes_certos
0,Liverpool,1603
1,Barcelona,1541
2,Inter Miami,1530
3,Club Brugge,1442
4,Bayern de Munique,1436
5,Al-Hilal,1410
6,Atlético,1381
7,Real Madrid,1345
8,Chelsea,1326
9,Fenerbahçe,1324


#### Passes em Profundidade

In [1959]:
passes_profundidade = (
    df
    .groupby("clube")["through_balls"]
    .sum()
    .reset_index(name="passes_profundidade")
    .sort_values("passes_profundidade", ascending=False)
    .reset_index(drop=True)
)

In [1960]:
passes_profundidade_2t = passes_profundidade.head(10)

passes_profundidade_2t

,clube,passes_profundidade
0,Manchester City,8
1,Inter Miami,6
2,PSV,5
3,Liverpool,5
4,Real Madrid,5
5,Alanyaspor,4
6,Club Brugge,4
7,Barcelona,4
8,Atlético Madrid,4
9,Real Sociedad,4


#### Entradas no Terço Final

In [1961]:
entradas_terco_final = (
    df
    .groupby("clube")["final_third_entries"]
    .sum()
    .reset_index(name="entradas_terco_final")
    .sort_values("entradas_terco_final", ascending=False)
    .reset_index(drop=True)
)

In [1962]:
entradas_terco_final_2t = entradas_terco_final.head(10)

entradas_terco_final_2t

,clube,entradas_terco_final
0,Once Caldas,228
1,Liverpool,218
2,Club Brugge,218
3,Real Madrid,215
4,Fenerbahçe,213
5,Atlético,205
6,Al-Hilal,203
7,Atlético Bucaramanga,200
8,Sanfrecce Hiroshima,197
9,América de Cali,197


#### Faltas Sofridas no Terço Final

In [1963]:
faltas_terco_final = (
    df
    .groupby("clube")["fouled_in_final_third"]
    .sum()
    .reset_index(name="faltas_terco_final")
    .sort_values("faltas_terco_final", ascending=False)
    .reset_index(drop=True)
)

In [1964]:
faltas_terco_final_2t = faltas_terco_final.head(10)

faltas_terco_final_2t

,clube,faltas_terco_final
0,Al-Hilal,15
1,Benfica,14
2,Vélez Sarsfield,14
3,Boyacá Chicó,13
4,Genk,12
5,Totora Real Oruro,12
6,San Antonio Bulo-Bulo,12
7,Once Caldas,12
8,Botafogo-SP,11
9,Nacional Potosí,11


#### Perda da Posse de Bola

In [1965]:
perda_posse = (
    df
    .groupby("clube")["dispossessed"]
    .sum()
    .reset_index(name="perda_posse")
    .sort_values("perda_posse", ascending=False)
    .reset_index(drop=True)
)

In [1966]:
perda_posse_2t = perda_posse.head(10)

perda_posse_2t

,clube,perda_posse
0,Benfica,40
1,Chelsea,37
2,Fatih Karagümrük,32
3,Atlético Tucumán,31
4,Club Brugge,30
5,Liverpool,30
6,Operário-PR,29
7,Kashiwa Reysol,29
8,Utrecht,28
9,Genk,28


#### Duelos Ganhos

In [1967]:
df["duels"] = (
    df["duels"]
    .astype(str)
    .str.replace("%", "", regex=False)
    .str.replace(",", ".", regex=False)
    .str.strip()
)

In [1968]:
df["duels"] = pd.to_numeric(df["duels"], errors="coerce")

In [1969]:
# Conta jogos por clube e filtra quem tem mais de 2
df_filtrado = (
    df.assign(jogos=df.groupby("clube")["clube"].transform("size"))
      .query("jogos > 2")
)

# Calcula a média de duelos ganhos apenas para os clubes com > 2 jogos
duelos_ganhos = (
    df_filtrado
    .groupby("clube", as_index=False)["duels"]
    .mean()
    .rename(columns={"duels": "duelos_ganhos"})
    .sort_values("duelos_ganhos", ascending=False)
    .reset_index(drop=True)
)

In [1970]:
duelos_ganhos_2t = duelos_ganhos.head(10)

duelos_ganhos_2t["duelos_ganhos"] = duelos_ganhos_2t["duelos_ganhos"].astype(int)

duelos_ganhos_2t

C:\Users\stefa\AppData\Local\Temp\ipykernel_19328\364493970.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duelos_ganhos_2t["duelos_ganhos"] = duelos_ganhos_2t["duelos_ganhos"].astype(int)


,clube,duelos_ganhos
0,Olympiacos,63
1,Auxerre,61
2,San Diego,61
3,Silkeborg,60
4,Larisa,59
5,Twente,59
6,Comerciantes Unidos,58
7,Deportivo Táchira,58
8,Lens,58
9,Al-Hilal,58


#### Cartões Amarelos

In [1971]:
amarelos = (
    df
    .groupby("clube")["yellow_cards"]
    .sum()
    .reset_index(name="amarelos")
    .sort_values("amarelos", ascending=False)
    .reset_index(drop=True)
)

In [1972]:
amarelos_2t = amarelos.head(10)

amarelos_2t

,clube,amarelos
0,Once Caldas,20
1,Blooming,17
2,Nacional Potosí,17
3,Deportivo Pasto,14
4,Vila Nova-GO,14
5,Atlético,14
6,Inter Miami,14
7,Chelsea,14
8,Cuiabá,13
9,Millonarios,13


#### Cartões Vermelhos

In [1973]:
vermelhos_2t_antes = (
    df
    .groupby("clube")["red_cards"]
    .sum()
    .reset_index(name="vermelhos")
    .sort_values("vermelhos", ascending=False)
    .reset_index(drop=True)
)

In [1974]:
vermelhos_2t = vermelhos_2t_antes[vermelhos_2t_antes["vermelhos"] > 1]

vermelhos_2t = vermelhos_2t_antes.head(10)

vermelhos_2t

,clube,vermelhos
0,Jorge Wilstermann,6
1,Blooming,5
2,Bolívar,3
3,Libertad,3
4,Aucas,3
5,Gençlerbirligi,2
6,Lazio,2
7,Bayer Leverkusen,2
8,Necaxa,2
9,Santos Laguna,2


### Partida Completa

#### Carregando a consulta

In [1975]:
conn = mysql.connector.connect(**config)

In [1976]:
query = """
SELECT
    p.data,
    ep.temporada,
    ep.clube,
    p.campeonato,
    p.pais,
    CASE
        WHEN ep.clube = p.casa THEN 'Casa'
        WHEN ep.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    ep.ball_possession,
    ep.expected_goals,
    ep.big_chances,
    ep.total_shots,
    ep.shots_on_target,
    ep.shots_inside_box,
    ep.hit_woodwork,
    ep.corner_kicks,
    ep.passes,
    ep.big_chances_scored,
    ep.accurate_passes,
    ep.through_balls,
    ep.final_third_entries,
    ep.fouled_in_final_third,
    ep.dispossessed,
    ep.duels,
    ep.yellow_cards
FROM
    estatisticas_partidas ep
    INNER JOIN partidas p ON ep.id_jogo = p.id_jogo
WHERE
    p.data >= DATE_SUB(DATE_FORMAT(CURDATE(), '%Y-%m-01'), INTERVAL 1 MONTH)
	AND p.data <  DATE_FORMAT(CURDATE(), '%Y-%m-01');;
"""

In [1977]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_19328\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [1978]:
conn.close()

In [1979]:
df = df[(df["campeonato"] != "Eliminatórias") & (df["campeonato"] != "Amistoso")]

#### Posse de Bola

Regra: o clube precisa ter realizado no mínimo 3 jogos no mês.

In [1980]:
df["ball_possession"] = (
    df["ball_possession"]
    .astype(str)
    .str.replace("%", "", regex=False)
    .str.replace(",", ".", regex=False)
    .str.strip()
)

In [1981]:
# Garante tipo numérico (se já estiver ok, mantém)
df["ball_possession"] = pd.to_numeric(df["ball_possession"], errors="coerce")

# Adiciona a contagem de jogos por clube e filtra quem tem > 2
df_filtrado = (
    df.assign(jogos=df.groupby("clube")["clube"].transform("size"))
      .query("jogos > 2")
)

# Calcula a posse de bola apenas para clubes com mais de 2 jogos
posse_bola = (
    df_filtrado
    .groupby("clube", as_index=False)["ball_possession"]
    .mean()
    .rename(columns={"ball_possession": "posse_bola"})
    .sort_values("posse_bola", ascending=False)
    .reset_index(drop=True)
)

In [1982]:
mais_posse_bola_pc = posse_bola.head(10)

mais_posse_bola_pc["posse_bola"] = mais_posse_bola_pc["posse_bola"].round(2)

mais_posse_bola_pc

C:\Users\stefa\AppData\Local\Temp\ipykernel_19328\1875601044.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mais_posse_bola_pc["posse_bola"] = mais_posse_bola_pc["posse_bola"].round(2)


,clube,posse_bola
0,Cerro Porteño,72.25
1,Barcelona,72.20
2,Shakhtar Donetsk,67.50
3,San Diego,67.33
4,PSG,67.25
5,Liverpool,65.67
6,Shelbourne,65.50
7,Arsenal,65.40
8,CRB,65.20
9,Fenerbahçe,65.20


#### xG Médio

In [1983]:
xg_medio = (
    df
    .groupby("clube")["expected_goals"]
    .mean()
    .reset_index(name="xg_medio")
    .sort_values("xg_medio", ascending=False)
    .reset_index(drop=True)
)

In [1984]:
xg_medio["xg_medio"] = xg_medio["xg_medio"].round(2)

xg_medio_pc = xg_medio.head(10)

xg_medio_pc

,clube,xg_medio
0,Nyva Ternopil,3.79
1,Bayern de Munique,3.13
2,Bodo/Glimt,3.01
3,Sporting,2.97
4,Ludogorets,2.92
5,PSV,2.68
6,Real Madrid,2.58
7,RB Leipzig,2.49
8,Molde,2.42
9,Vancouver,2.42


#### Finalizações

In [1985]:
finalizacoes = (
    df
    .groupby("clube")["total_shots"]
    .sum()
    .reset_index(name="finalizacoes")
    .sort_values("finalizacoes", ascending=False)
    .reset_index(drop=True)
)

In [1986]:
finalizacoes_pc = finalizacoes.head(10)

finalizacoes_pc

,clube,finalizacoes
0,Al-Hilal,116
1,Sanfrecce Hiroshima,114
2,Real Madrid,110
3,Bayern de Munique,108
4,Club Brugge,106
5,Liverpool,104
6,Barcelona,103
7,América de Cali,99
8,Cuiabá,98
9,Avaí,97


#### Finalizações Certas

In [1987]:
finalizacoes_certas = (
    df
    .groupby("clube")["shots_on_target"]
    .sum()
    .reset_index(name="finalizacoes_certas")
    .sort_values("finalizacoes_certas", ascending=False)
    .reset_index(drop=True)
)

In [1988]:
finalizacoes_certas_pc = finalizacoes_certas.head(10)

finalizacoes_certas_pc

,clube,finalizacoes_certas
0,Bayern de Munique,55
1,Real Madrid,50
2,Al-Hilal,44
3,Barcelona,44
4,Inter Miami,44
5,Sanfrecce Hiroshima,42
6,Atlético Madrid,41
7,Al-Nassr,39
8,Aurora,39
9,Bolívar,38


#### Finalizações Dentro da Área

In [1989]:
finalizacoes_area = (
    df
    .groupby("clube")["shots_inside_box"]
    .sum()
    .reset_index(name="finalizacoes_area")
    .sort_values("finalizacoes_area", ascending=False)
    .reset_index(drop=True)
)

In [1990]:
finalizacoes_area_pc = finalizacoes_area.head(10)

finalizacoes_area_pc

,clube,finalizacoes_area
0,Al-Hilal,81
1,Club Brugge,78
2,Real Madrid,77
3,Sanfrecce Hiroshima,73
4,Liverpool,72
5,Bayern de Munique,71
6,América de Cali,67
7,Barcelona,67
8,Internazionale,65
9,Al-Nassr,62


#### Bolas na Trave

In [1991]:
bola_trave = (
    df
    .groupby("clube")["hit_woodwork"]
    .sum()
    .reset_index(name="bola_trave")
    .sort_values("bola_trave", ascending=False)
    .reset_index(drop=True)
)

In [1992]:
bola_trave_pc = bola_trave.head(10)

bola_trave_pc

,clube,bola_trave
0,Milan,8
1,Sanfrecce Hiroshima,6
2,Bayern de Munique,6
3,Coritiba,6
4,Oriente Petrolero,6
5,Real Madrid,6
6,Inter Miami,5
7,Atlético,5
8,Nacional Potosí,5
9,São Paulo,5


#### Grandes Chances

In [1993]:
grandes_chances = (
    df
    .groupby("clube")["big_chances"]
    .sum()
    .reset_index(name="grandes_chances")
    .sort_values("grandes_chances", ascending=False)
    .reset_index(drop=True)
)

In [1994]:
grandes_chances_pc = grandes_chances.head(10)

grandes_chances_pc

,clube,grandes_chances
0,Atlético Madrid,29
1,Bayern de Munique,29
2,Internazionale,27
3,Real Madrid,27
4,Al-Nassr,25
5,Liverpool,24
6,Al-Hilal,23
7,Inter Miami,23
8,Go Ahead Eagles,22
9,PSV,20


#### Grandes Chances Aproveitadas

In [1995]:
grandes_chances_aproveitadas = (
    df
    .groupby("clube")["big_chances_scored"]
    .sum()
    .reset_index(name="grandes_chances_aproveitadas")
    .sort_values("grandes_chances_aproveitadas", ascending=False)
    .reset_index(drop=True)
)

In [1996]:
grandes_chances_aproveitadas_pc = grandes_chances_aproveitadas.head(10)

grandes_chances_aproveitadas_pc

,clube,grandes_chances_aproveitadas
0,Bayern de Munique,13
1,Atlético Madrid,13
2,Real Madrid,12
3,Al-Hilal,11
4,Los Angeles,10
5,Inter Miami,10
6,Palmeiras,10
7,Benfica,9
8,Barcelona,9
9,Internazionale,9


#### Escanteios

In [1997]:
escanteios = (
    df
    .groupby("clube")["corner_kicks"]
    .sum()
    .reset_index(name="escanteios")
    .sort_values("escanteios", ascending=False)
    .reset_index(drop=True)
)

In [1998]:
escanteios_pc = escanteios.head(10)

escanteios_pc

,clube,escanteios
0,Tottenham,50
1,Sanfrecce Hiroshima,49
2,Al-Ahli,43
3,Novorizontino,42
4,Liverpool,41
5,Atlético Madrid,40
6,Al-Hilal,40
7,Urawa Red Diamonds,39
8,Arsenal,39
9,Al-Nassr,39


#### Passes

In [1999]:
passes = (
    df
    .groupby("clube")["passes"]
    .sum()
    .reset_index(name="passes")
    .sort_values("passes", ascending=False)
    .reset_index(drop=True)
)

In [2000]:
passes_pc = passes.head(10)

passes_pc

,clube,passes
0,Sanfrecce Hiroshima,3814
1,Liverpool,3637
2,Barcelona,3485
3,Club Brugge,3411
4,Inter Miami,3410
5,Urawa Red Diamonds,3386
6,Real Madrid,3318
7,Bayern de Munique,3279
8,Al-Hilal,3203
9,Kashiwa Reysol,3182


#### Passes Certos

In [2001]:
passes_certos = (
    df
    .groupby("clube")["accurate_passes"]
    .sum()
    .reset_index(name="passes_certos")
    .sort_values("passes_certos", ascending=False)
    .reset_index(drop=True)
)

In [2002]:
passes_certos_pc = passes_certos.head(10)

passes_certos_pc

,clube,passes_certos
0,Liverpool,3204
1,Sanfrecce Hiroshima,3159
2,Barcelona,3143
3,Inter Miami,3020
4,Club Brugge,2971
5,Bayern de Munique,2944
6,Real Madrid,2935
7,Urawa Red Diamonds,2893
8,Al-Hilal,2816
9,Chelsea,2779


#### Passes em Profundidade

In [2003]:
passes_profundidade = (
    df
    .groupby("clube")["through_balls"]
    .sum()
    .reset_index(name="passes_profundidade")
    .sort_values("passes_profundidade", ascending=False)
    .reset_index(drop=True)
)

In [2004]:
passes_profundidade_pc = passes_profundidade.head(10)

passes_profundidade_pc

,clube,passes_profundidade
0,Inter Miami,12
1,Yokohama,12
2,Liverpool,11
3,Manchester City,10
4,PSG,10
5,Shakhtar Donetsk,9
6,Urawa Red Diamonds,9
7,Club Brugge,9
8,Chicago Fire,9
9,Barcelona,8


#### Entradas no Terço Final

In [2005]:
entradas_terco_final = (
    df
    .groupby("clube")["final_third_entries"]
    .sum()
    .reset_index(name="entradas_terco_final")
    .sort_values("entradas_terco_final", ascending=False)
    .reset_index(drop=True)
)

In [2006]:
entradas_terco_final_pc = entradas_terco_final.head(10)

entradas_terco_final_pc

,clube,entradas_terco_final
0,Once Caldas,466
1,Real Madrid,449
2,Independiente Del Valle,437
3,Liverpool,416
4,Sanfrecce Hiroshima,410
5,Al-Hilal,408
6,Club Brugge,407
7,Barcelona,406
8,Atlético Bucaramanga,389
9,Nacional Potosí,385


#### Faltas Sofridas no Terço Final

In [2007]:
faltas_terco_final = (
    df
    .groupby("clube")["fouled_in_final_third"]
    .sum()
    .reset_index(name="faltas_terco_final")
    .sort_values("faltas_terco_final", ascending=False)
    .reset_index(drop=True)
)

In [2008]:
faltas_terco_final_pc = faltas_terco_final.head(10)

faltas_terco_final_pc

,clube,faltas_terco_final
0,Al-Hilal,26
1,Totora Real Oruro,21
2,Boyacá Chicó,21
3,Benfica,21
4,San Antonio Bulo-Bulo,20
5,Genk,20
6,Athletico-PR,20
7,Alianza Universidad,19
8,Guabirá,19
9,Junior Barranquilla,19


#### Perda da Posse de Bola

In [2009]:
perda_posse = (
    df
    .groupby("clube")["dispossessed"]
    .sum()
    .reset_index(name="perda_posse")
    .sort_values("perda_posse", ascending=False)
    .reset_index(drop=True)
)

In [2010]:
perda_posse_pc = perda_posse.head(10)

perda_posse_pc

,clube,perda_posse
0,Benfica,72
1,Aston Villa,67
2,Kashiwa Reysol,62
3,Urawa Red Diamonds,58
4,Remo,56
5,Liverpool,56
6,Bahia,55
7,Genk,54
8,Atlético,53
9,Club Brugge,53


#### Duelos Ganhos

In [2011]:
df["duels"] = (
    df["duels"]
    .astype(str)
    .str.replace("%", "", regex=False)
    .str.replace(",", ".", regex=False)
    .str.strip()
)

In [2012]:
df["duels"] = pd.to_numeric(df["duels"], errors="coerce")

In [2013]:
# Conta jogos por clube e filtra quem tem mais de 2
df_filtrado = (
    df.assign(jogos=df.groupby("clube")["clube"].transform("size"))
      .query("jogos > 2")
)

# Calcula a média de duelos ganhos apenas para os clubes com > 2 jogos
duelos_ganhos = (
    df_filtrado
    .groupby("clube", as_index=False)["duels"]
    .mean()
    .rename(columns={"duels": "duelos_ganhos"})
    .sort_values("duelos_ganhos", ascending=False)
    .reset_index(drop=True)
)

In [2014]:
duelos_ganhos_pc = duelos_ganhos.head(10)

duelos_ganhos_pc["duelos_ganhos"] = duelos_ganhos_pc["duelos_ganhos"].astype(int)

duelos_ganhos_pc

C:\Users\stefa\AppData\Local\Temp\ipykernel_19328\1749527485.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duelos_ganhos_pc["duelos_ganhos"] = duelos_ganhos_pc["duelos_ganhos"].astype(int)


,clube,duelos_ganhos
0,Nagoya Grampus,61
1,Sligo Rovers,57
2,Huracán,57
3,Universitario,56
4,Instituto,56
5,Alverca,56
6,Tokyo,56
7,Cincinnati,56
8,Alianza Valledupar,56
9,Anzoátegui,55


#### Cartões Amarelos

In [2015]:
amarelos = (
    df
    .groupby("clube")["yellow_cards"]
    .sum()
    .reset_index(name="amarelos")
    .sort_values("amarelos", ascending=False)
    .reset_index(drop=True)
)

In [2016]:
amarelos_pc = amarelos.head(10)

amarelos_pc

,clube,amarelos
0,Once Caldas,27
1,Deportivo Pasto,26
2,Vila Nova-GO,23
3,Asteras Aktor,22
4,Atlético,21
5,Nacional Potosí,21
6,Oriente Petrolero,21
7,Millonarios,21
8,Totora Real Oruro,20
9,Inter Miami,20


#### Cartões Vermelhos

In [2017]:
vermelhos_pc = vermelhos_antes.merge(vermelhos_2t_antes, how="inner", on="clube")

vermelhos_pc.head()

,clube,vermelhos_x,vermelhos_y
0,Vélez Sarsfield,2,1
1,El Nacional,2,0
2,Al-Shabab,2,0
3,Deportivo Táchira,2,0
4,Cruz Azul,2,1


In [2018]:
vermelhos_pc["vermelhos"] = vermelhos_pc["vermelhos_x"] + vermelhos_pc["vermelhos_y"]

vermelhos_pc = vermelhos_pc[["clube", "vermelhos"]]

vermelhos_pc.head()

,clube,vermelhos
0,Vélez Sarsfield,3
1,El Nacional,2
2,Al-Shabab,2
3,Deportivo Táchira,2
4,Cruz Azul,3


In [2019]:
vermelhos_pc = vermelhos_pc[vermelhos_pc["vermelhos"] > 1]

vermelhos_pc = vermelhos_pc.sort_values("vermelhos", ascending=False)

vermelhos_pc = vermelhos_pc.head(10)

vermelhos_pc

,clube,vermelhos
514,Jorge Wilstermann,6
330,Blooming,5
0,Vélez Sarsfield,3
38,River Plate,3
384,Aucas,3
106,Libertad,3
325,Bolívar,3
25,Necaxa,3
15,Sporting Cristal,3
7,Aurora,3


## Individual

### Jogadores

#### Consulta no banco de dados

In [2020]:
conn = mysql.connector.connect(**config)

In [2021]:
query = """
SELECT
    p.data,
    p.id_jogo,
    ej.temporada,
    CASE
        WHEN ej.team = p.casa THEN 'Casa'
        WHEN ej.team = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    ej.player_id,
    ej.name,
    CASE
        WHEN ej.position = 'G' THEN 'Goleiro'
        WHEN ej.position = 'D' THEN 'Defensor'
        WHEN ej.position = 'M' THEN 'Meio-campo'
        WHEN ej.position = 'F' THEN 'Atacante'
        ELSE 'Verificar'
    END AS position,
    ej.team,
    ej.country,
    ej.rating,
    ej.minutes_played,
    ej.goals,
    ej.goal_assist,
    ej.big_chance_created,
    ej.key_pass,
    ej.expected_goals
FROM
    estatisticas_jogadores ej
    INNER JOIN partidas p ON ej.id_jogo = p.id_jogo
WHERE
    ej.minutes_played > 0
    AND data >= DATE_SUB(DATE_FORMAT(CURDATE(), '%Y-%m-01'), INTERVAL 1 MONTH)
	AND data <  DATE_FORMAT(CURDATE(), '%Y-%m-01');
"""

In [2022]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_19328\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [2023]:
conn.close()

In [2024]:
df.head(5)

,data,id_jogo,temporada,mando,player_id,name,position,team,country,rating,minutes_played,goals,goal_assist,big_chance_created,key_pass,expected_goals
0,2025-09-01,13616309,2025,Casa,166279,Léo Vieira,Goleiro,Chapecoense,Brazil,6.6,90,0,0,0,0,0.0
1,2025-09-01,13616309,2025,Casa,863292,Victor Caetano,Defensor,Chapecoense,Brazil,6.5,15,0,0,0,0,0.0
2,2025-09-01,13616309,2025,Casa,1122967,Eduardo Doma,Defensor,Chapecoense,Brazil,6.5,90,0,0,0,0,0.0
3,2025-09-01,13616309,2025,Casa,867913,João Paulo,Defensor,Chapecoense,Brazil,7.1,90,0,0,0,0,0.0
4,2025-09-01,13616309,2025,Casa,1047680,Gabriel Inocêncio,Defensor,Chapecoense,Brazil,6.9,85,0,0,0,1,0.0


#### Mais minutos

In [2025]:
mais_minutos = (
    df.groupby(["player_id", "name", "position"], as_index=False)
      .agg(minutos_jogados=("minutes_played", "sum"))
      .sort_values("minutos_jogados", ascending=False)
)

##### Goleiros

In [2026]:
minutos_goleiros = mais_minutos[mais_minutos["position"] == "Goleiro"]

minutos_goleiros.head(3)

,player_id,name,position,minutos_jogados
2209,360938,Bono,Goleiro,720
370,69768,Jan Oblak,Goleiro,720
373,70988,Thibaut Courtois,Goleiro,720


In [2027]:
minutos_goleiros = minutos_goleiros.merge(df, how="inner", on="player_id")

minutos_goleiros = minutos_goleiros[["name_x", "position_x", "team", "minutos_jogados"]]

minutos_goleiros.head(3)

,name_x,position_x,team,minutos_jogados
0,Bono,Goleiro,Marrocos,720
1,Bono,Goleiro,Marrocos,720
2,Bono,Goleiro,Al-Hilal,720


In [2028]:
minutos_goleiros = (
    minutos_goleiros
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "minutos_jogados": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2029]:
minutos_goleiros = minutos_goleiros.sort_values("minutos_jogados", ascending=False)

minutos_goleiros = minutos_goleiros.head(10)

minutos_goleiros

,name,position,team,minutos_jogados
383,Jan Oblak,Goleiro,"Atlético Madrid, Eslovénia",720
58,Anatoliy Trubin,Goleiro,"Benfica, Ucrânia",720
830,Thibaut Courtois,Goleiro,"Bélgica, Real Madrid",720
124,Bono,Goleiro,"Al-Hilal, Marrocos",720
853,Unai Simón,Goleiro,"Athletic Bilbao, Espanha",630
147,Carlos Lampe,Goleiro,"Bolívar, Bolívia",630
854,Uğurcan Çakır,Goleiro,"Galatasaray, Turquia",630
588,Matthew Freese,Goleiro,"Estados Unidos, New York City",607
49,Alisson,Goleiro,"Brasil, Criciúma, Liverpool",596
501,Louis Yamaguchi,Goleiro,Kawasaki Frontale,570


##### Defesores

In [2030]:
minutos_defensores = mais_minutos[mais_minutos["position"] == "Defensor"]

minutos_defensores.head(3)

,player_id,name,position,minutos_jogados
2699,787751,Robin Le Normand,Defensor,662
389,74915,Nicolás Otamendi,Defensor,661
1196,187205,Milan Škriniar,Defensor,630


In [2031]:
minutos_defensores = minutos_defensores.merge(df, how="inner", on="player_id")

minutos_defensores = minutos_defensores[["name_x", "position_x", "team", "minutos_jogados"]]

minutos_defensores.head(3)

,name_x,position_x,team,minutos_jogados
0,Robin Le Normand,Defensor,Espanha,662
1,Robin Le Normand,Defensor,Espanha,662
2,Robin Le Normand,Defensor,Atlético Madrid,662


In [2032]:
minutos_defensores = (
    minutos_defensores
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "minutos_jogados": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2033]:
minutos_defensores = minutos_defensores.sort_values("minutos_jogados", ascending=False)

minutos_defensores = minutos_defensores.head(10)

minutos_defensores

,name,position,team,minutos_jogados
3787,Robin Le Normand,Defensor,"Atlético Madrid, Espanha",662
3372,Nicolás Otamendi,Defensor,"Argentina, Benfica",661
461,Arthur Theate,Defensor,"Bélgica, Eintracht Frankfurt",630
3156,Milan Škriniar,Defensor,"Eslováquia, Fenerbahçe",630
4410,Virgil van Dijk,Defensor,"Holanda, Liverpool",630
3608,Pedro Porro,Defensor,"Espanha, Tottenham",628
2495,Konrad Laimer,Defensor,"Bayern de Munique, Áustria",623
1772,Ibrahima Konaté,Defensor,"França, Liverpool",611
3018,Matty Cash,Defensor,"Aston Villa, Polônia",585
1674,Hassan Al Tambakti,Defensor,"Al-Hilal, Arábia Saudita",585


##### Meio-campo

In [2034]:
minutos_meias = mais_minutos[mais_minutos["position"] == "Meio-campo"]

minutos_meias.head(3)

,player_id,name,position,minutos_jogados
1491,249399,Rodrigo de Paul,Meio-campo,698
8626,1015292,Richard Ríos,Meio-campo,677
9023,1031259,Christos Tzolis,Meio-campo,668


In [2035]:
minutos_meias = minutos_meias.merge(df, how="inner", on="player_id")

minutos_meias = minutos_meias[["name_x", "position_x", "team", "minutos_jogados"]]

minutos_meias.head(3)

,name_x,position_x,team,minutos_jogados
0,Rodrigo de Paul,Meio-campo,Argentina,698
1,Rodrigo de Paul,Meio-campo,Argentina,698
2,Rodrigo de Paul,Meio-campo,Inter Miami,698


In [2036]:
minutos_meias = (
    minutos_meias
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "minutos_jogados": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2037]:
minutos_meias = minutos_meias.sort_values("minutos_jogados", ascending=False)

minutos_meias = minutos_meias.head(10)

minutos_meias

,name,position,team,minutos_jogados
4797,Rodrigo de Paul,Meio-campo,"Argentina, Inter Miami",698
4706,Richard Ríos,Meio-campo,"Benfica, Colômbia",677
950,Christos Tzolis,Meio-campo,"Club Brugge, Grécia",668
2062,Hans Vanaken,Meio-campo,"Bélgica, Club Brugge",649
160,Alan Franco,Meio-campo,"Atlético, Equador",640
1343,Dominik Szoboszlai,Meio-campo,"Hungria, Liverpool",630
5012,Satoshi Tanaka,Meio-campo,Sanfrecce Hiroshima,629
607,Aurélien Tchouaméni,Meio-campo,"França, Real Madrid",616
1909,Georgiy Sudakov,Meio-campo,"Benfica, Ucrânia",613
2647,John McGinn,Meio-campo,"Aston Villa, Escócia",612


##### Atacantes

In [2038]:
minutos_atacantes = mais_minutos[mais_minutos["position"] == "Atacante"]

minutos_atacantes.head(3)

,player_id,name,position,minutos_jogados
3729,826643,Kylian Mbappé,Atacante,702
3011,795236,Vangelis Pavlidis,Atacante,691
15,12994,Lionel Messi,Atacante,630


In [2039]:
minutos_atacantes = minutos_atacantes.merge(df, how="inner", on="player_id")

minutos_atacantes = minutos_atacantes[["name_x", "position_x", "team", "minutos_jogados"]]

minutos_atacantes.head(3)

,name_x,position_x,team,minutos_jogados
0,Kylian Mbappé,Atacante,França,702
1,Kylian Mbappé,Atacante,França,702
2,Kylian Mbappé,Atacante,Real Madrid,702


In [2040]:
minutos_atacantes = (
    minutos_atacantes
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "minutos_jogados": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2041]:
minutos_atacantes = minutos_atacantes.sort_values("minutos_jogados", ascending=False)

minutos_atacantes = minutos_atacantes.head(10)

minutos_atacantes

,name,position,team,minutos_jogados
1618,Kylian Mbappé,Atacante,"França, Real Madrid",702
2743,Vangelis Pavlidis,Atacante,"Benfica, Grécia",691
1668,Lionel Messi,Atacante,"Argentina, Inter Miami",630
2142,Nikola Krstović,Atacante,"Atalanta, Montenegro",571
1722,Luis Díaz,Atacante,"Bayern de Munique, Colômbia",562
1099,Harry Kane,Atacante,"Bayern de Munique, Inglaterra",551
82,Akito Suzuki,Atacante,Shonan Bellmare,540
2886,Yoshan Valois,Atacante,Deportivo Pasto,534
931,Florian Wirtz,Atacante,"Alemanha, Liverpool",534
2945,Óscar Villalba,Atacante,Independiente/BOL,524


##### Minutos Geral

In [2042]:
mais_minutos = mais_minutos.merge(df, how="inner", on="player_id")

mais_minutos = mais_minutos[["name_x", "position_x", "team", "minutos_jogados"]]

mais_minutos.head(3)

,name_x,position_x,team,minutos_jogados
0,Bono,Goleiro,Marrocos,720
1,Bono,Goleiro,Marrocos,720
2,Bono,Goleiro,Al-Hilal,720


In [2043]:
mais_minutos = (
    mais_minutos
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "minutos_jogados": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2044]:
mais_minutos = mais_minutos.sort_values("minutos_jogados", ascending=False)

mais_minutos = mais_minutos.head(10)

mais_minutos

,name,position,team,minutos_jogados
942,Anatoliy Trubin,Goleiro,"Benfica, Ucrânia",720
1827,Bono,Goleiro,"Al-Hilal, Marrocos",720
6068,Jan Oblak,Goleiro,"Atlético Madrid, Eslovénia",720
13197,Thibaut Courtois,Goleiro,"Bélgica, Real Madrid",720
7899,Kylian Mbappé,Atacante,"França, Real Madrid",702
11927,Rodrigo de Paul,Meio-campo,"Argentina, Inter Miami",698
13640,Vangelis Pavlidis,Atacante,"Benfica, Grécia",691
11722,Richard Ríos,Meio-campo,"Benfica, Colômbia",677
2404,Christos Tzolis,Meio-campo,"Club Brugge, Grécia",668
11835,Robin Le Normand,Defensor,"Atlético Madrid, Espanha",662


#### Artilharia

In [2045]:
artilharia = (
    df.groupby(["player_id", "name", "position"], as_index=False)
      .agg(gols=("goals", "sum"))
      .sort_values("gols", ascending=False)
)

In [2046]:
artilharia = artilharia[artilharia["gols"] > 0]

##### Goleiros

In [2047]:
gols_goleiros = artilharia[artilharia["position"] == "Goleiro"].head(10)

gols_goleiros

,player_id,name,position,gols
1434,243509,Tiago Volpi,Goleiro,1


##### Defensores

In [2048]:
gols_defensores = artilharia[artilharia["position"] == "Defensor"]

gols_defensores.head(3)

,player_id,name,position,gols
2706,788027,Theo Hernández,Defensor,4
3931,833956,Matty Cash,Defensor,3
2476,581064,Stefan Velkov,Defensor,3


In [2049]:
gols_defensores = gols_defensores.merge(df, how="inner", on="player_id")

gols_defensores = gols_defensores[["name_x", "position_x", "team", "gols"]]

gols_defensores.head(3)

,name_x,position_x,team,gols
0,Theo Hernández,Defensor,França,4
1,Theo Hernández,Defensor,Al-Hilal,4
2,Theo Hernández,Defensor,Al-Hilal,4


In [2050]:
gols_defensores = (
    gols_defensores
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "gols": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2051]:
gols_defensores = gols_defensores.sort_values("gols", ascending=False)

gols_defensores = gols_defensores.head(10)

gols_defensores

,name,position,team,gols
358,Theo Hernández,Defensor,"Al-Hilal, França",4
355,Stefan Velkov,Defensor,Vejle,3
251,Matty Cash,Defensor,"Aston Villa, Polônia",3
347,Sebastián Valencia,Defensor,Fortaleza/COL,2
53,Carlos Harvey,Defensor,"Minnesota United, Panamá",2
183,João Cancelo,Defensor,"Al-Hilal, Portugal",2
69,Daniel Høegh,Defensor,Randers,2
38,Bart van Rooij,Defensor,Twente,2
239,Marcos Llorente,Defensor,Atlético Madrid,2
264,Micky van de Ven,Defensor,"Holanda, Tottenham",2


##### Meio-campo

In [2052]:
gols_meias = artilharia[artilharia["position"] == "Meio-campo"]

gols_meias.head(3)

,player_id,name,position,gols
2562,592010,Mikel Merino,Meio-campo,5
6112,921347,Maksim Glushenkov,Meio-campo,5
2801,790108,Alejandro Zendejas,Meio-campo,5


In [2053]:
gols_meias = gols_meias.merge(df, how="inner", on="player_id")

gols_meias = gols_meias[["name_x", "position_x", "team", "gols"]]

gols_meias.head(3)

,name_x,position_x,team,gols
0,Mikel Merino,Meio-campo,Espanha,5
1,Mikel Merino,Meio-campo,Espanha,5
2,Mikel Merino,Meio-campo,Arsenal,5


In [2054]:
gols_meias = (
    gols_meias
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "gols": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2055]:
gols_meias = gols_meias.sort_values("gols", ascending=False)

gols_meias = gols_meias.head(10)

gols_meias

,name,position,team,gols
636,Maksim Glushenkov,Meio-campo,"Russia, Zenit",5
28,Alejandro Zendejas,Meio-campo,"América/MEX, Estados Unidos",5
956,Tatsuya Ito,Meio-campo,Kawasaki Frontale,5
544,Kevin De Bruyne,Meio-campo,"Bélgica, Napoli",5
722,Mikel Merino,Meio-campo,"Arsenal, Espanha",5
396,Ismael Saibari,Meio-campo,"Marrocos, PSV",4
35,Alex Caceres,Meio-campo,Jorge Wilstermann,4
139,Carlos Arrua,Meio-campo,Nacional/PAR,4
852,Rafa Silva,Meio-campo,Besiktas,4
157,Christian Pulišić,Meio-campo,"Estados Unidos, Milan",4


##### Atacantes

In [2056]:
gols_atacantes = artilharia[artilharia["position"] == "Atacante"]

gols_atacantes.head(3)

,player_id,name,position,gols
3729,826643,Kylian Mbappé,Atacante,12
4045,839956,Erling Haaland,Atacante,12
568,108579,Harry Kane,Atacante,12


In [2057]:
gols_atacantes = gols_atacantes.merge(df, how="inner", on="player_id")

gols_atacantes = gols_atacantes[["name_x", "position_x", "team", "gols"]]

gols_atacantes.head(3)

,name_x,position_x,team,gols
0,Kylian Mbappé,Atacante,França,12
1,Kylian Mbappé,Atacante,França,12
2,Kylian Mbappé,Atacante,Real Madrid,12


In [2058]:
gols_atacantes = (
    gols_atacantes
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "gols": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2059]:
gols_atacantes = gols_atacantes.sort_values("gols", ascending=False)

gols_atacantes = gols_atacantes.head(10)

gols_atacantes

,name,position,team,gols
361,Harry Kane,Atacante,"Bayern de Munique, Inglaterra",12
264,Erling Haaland,Atacante,"Manchester City, Noruega",12
538,Kylian Mbappé,Atacante,"França, Real Madrid",12
368,Heung-min Son,Atacante,"Coreia do Sul, Los Angeles",9
196,Denis Bouanga,Atacante,Los Angeles,8
573,Luis Javier Suárez,Atacante,"Colômbia, Sporting",7
552,Lionel Messi,Atacante,"Argentina, Inter Miami",7
492,Julián Álvarez,Atacante,"Argentina, Atlético Madrid",6
867,Vangelis Pavlidis,Atacante,"Benfica, Grécia",6
932,Óscar Villalba,Atacante,Independiente/BOL,6


##### Artilharia Geral

In [2060]:
artilharia = artilharia.merge(df, how="inner", on="player_id")

artilharia = artilharia[["name_x", "position_x", "team", "gols"]]

artilharia.head(3)

,name_x,position_x,team,gols
0,Kylian Mbappé,Atacante,França,12
1,Kylian Mbappé,Atacante,França,12
2,Kylian Mbappé,Atacante,Real Madrid,12


In [2061]:
artilharia = (
    artilharia
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "gols": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2062]:
artilharia = artilharia.sort_values("gols", ascending=False)

artilharia = artilharia.head(10)

artilharia

,name,position,team,gols
1319,Kylian Mbappé,Atacante,"França, Real Madrid",12
629,Erling Haaland,Atacante,"Manchester City, Noruega",12
861,Harry Kane,Atacante,"Bayern de Munique, Inglaterra",12
880,Heung-min Son,Atacante,"Coreia do Sul, Los Angeles",9
475,Denis Bouanga,Atacante,Los Angeles,8
1421,Luis Javier Suárez,Atacante,"Colômbia, Sporting",7
1361,Lionel Messi,Atacante,"Argentina, Inter Miami",7
402,Cristiano Ronaldo,Atacante,"Al-Nassr, Portugal",6
2246,Vangelis Pavlidis,Atacante,"Benfica, Grécia",6
1209,Julián Álvarez,Atacante,"Argentina, Atlético Madrid",6


#### Assistências

In [2063]:
assistencias = (
    df.groupby(["player_id", "name", "position"], as_index=False)
      .agg(assistencias=("goal_assist", "sum"))
      .sort_values("assistencias", ascending=False)
)

In [2064]:
assistencias = assistencias[assistencias["assistencias"] > 0]

##### Goleiros

In [2065]:
assist_goleiros = assistencias[assistencias["position"] == "Goleiro"]

assist_goleiros.head(3)

,player_id,name,position,assistencias
3584,822590,Franco Petroli,Goleiro,1
2504,584722,Alexander Nübel,Goleiro,1
2367,548688,Jacob Karlstrøm,Goleiro,1


In [2066]:
assist_goleiros = assist_goleiros.merge(df, how="inner", on="player_id")

assist_goleiros = assist_goleiros[["name_x", "position_x", "team", "assistencias"]]

assist_goleiros.head(3)

,name_x,position_x,team,assistencias
0,Franco Petroli,Goleiro,Godoy Cruz,1
1,Franco Petroli,Goleiro,Godoy Cruz,1
2,Franco Petroli,Goleiro,Godoy Cruz,1


In [2067]:
assist_goleiros = (
    assist_goleiros
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "assistencias": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2068]:
assist_goleiros = assist_goleiros.sort_values("assistencias", ascending=False)

assist_goleiros = assist_goleiros.head(10)

assist_goleiros

,name,position,team,assistencias
0,Alexander Nübel,Goleiro,Stuttgart,1
1,André Onana,Goleiro,Trabzonspor,1
2,Conor Brann,Goleiro,Cork City,1
3,Franco Petroli,Goleiro,Godoy Cruz,1
4,Jacob Karlstrøm,Goleiro,Molde,1
5,Johan Gutiérrez,Goleiro,Universitario de Vinto,1
6,Lucas Cháves,Goleiro,Panetolikos,1
7,Ronaldo,Goleiro,Bahia,1
8,Thibaut Courtois,Goleiro,"Bélgica, Real Madrid",1


##### Defensores

In [2069]:
assist_defensor = assistencias[assistencias["position"] == "Defensor"]

assist_defensor.head(3)

,player_id,name,position,assistencias
139,35199,Jordi Alba,Defensor,4
10083,1113368,Rodrigo Huescas,Defensor,4
1458,246979,Enrique Flores,Defensor,3


In [2070]:
assist_defensor = assist_defensor.merge(df, how="inner", on="player_id")

assist_defensor = assist_defensor[["name_x", "position_x", "team", "assistencias"]]

assist_defensor.head(3)

,name_x,position_x,team,assistencias
0,Jordi Alba,Defensor,Inter Miami,4
1,Jordi Alba,Defensor,Inter Miami,4
2,Jordi Alba,Defensor,Inter Miami,4


In [2071]:
assist_defensor = (
    assist_defensor
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "assistencias": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2072]:
assist_defensor = assist_defensor.sort_values("assistencias", ascending=False)

assist_defensor = assist_defensor.head(10)

assist_defensor

,name,position,team,assistencias
187,Jordi Alba,Defensor,Inter Miami,4
350,Rodrigo Huescas,Defensor,"Copenhague, México",4
47,Artem Smoliakov,Defensor,Los Angeles,3
211,Julian Ryerson,Defensor,"Borussia Dortmund, Noruega",3
84,Daniel Lovitz,Defensor,Nashville,3
36,Andreas Poulsen,Defensor,Silkeborg,3
110,Enrique Flores,Defensor,San Antonio Bulo-Bulo,3
151,Hampus Skoglund,Defensor,Hammarby,3
191,Joseph Scally,Defensor,Borussia M'gladbach,2
54,Blás Riveros,Defensor,Cerro Porteño,2


##### Meio-campo

In [2074]:
assist_meias = assistencias[assistencias["position"] == "Meio-campo"]

assist_meias.head(3)

,player_id,name,position,assistencias
3575,822515,Evander,Meio-campo,5
1610,280441,Kingsley Coman,Meio-campo,5
5752,901891,Alexis Saelemaekers,Meio-campo,5


In [2075]:
assist_meias = assist_meias.merge(df, how="inner", on="player_id")

assist_meias = assist_meias[["name_x", "position_x", "team", "assistencias"]]

assist_meias.head(3)

,name_x,position_x,team,assistencias
0,Evander,Meio-campo,Cincinnati,5
1,Evander,Meio-campo,Cincinnati,5
2,Evander,Meio-campo,Cincinnati,5


In [2076]:
assist_meias = (
    assist_meias
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "assistencias": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2077]:
assist_meias = assist_meias.sort_values("assistencias", ascending=False)

assist_meias = assist_meias.head(10)

assist_meias

,name,position,team,assistencias
224,Evander,Meio-campo,Cincinnati,5
42,Alexis Saelemaekers,Meio-campo,"Bélgica, Milan",5
454,Kingsley Coman,Meio-campo,"Al-Nassr, França",5
143,Cristian Barco,Meio-campo,Guabirá,5
57,André Franco,Meio-campo,Chicago Fire,4
71,Arda Güler,Meio-campo,"Real Madrid, Turquia",4
334,Iván Colman,Meio-campo,Cusco,4
43,Alexis Vega,Meio-campo,"México, Toluca",4
466,Lamine Yamal,Meio-campo,"Barcelona, Espanha",4
296,Gustavo Scarpa,Meio-campo,Atlético,4


##### Atacantes

In [2078]:
assist_atacantes = assistencias[assistencias["position"] == "Atacante"]

assist_atacantes.head(3)

,player_id,name,position,assistencias
9726,1096455,William Álvarez,Atacante,5
6991,958592,Luis Gómez,Atacante,4
3011,795236,Vangelis Pavlidis,Atacante,4


In [2079]:
assist_atacantes = assist_atacantes.merge(df, how="inner", on="player_id")

assist_atacantes = assist_atacantes[["name_x", "position_x", "team", "assistencias"]]

assist_atacantes.head(3)

,name_x,position_x,team,assistencias
0,William Álvarez,Atacante,Nacional Potosí,5
1,William Álvarez,Atacante,Nacional Potosí,5
2,William Álvarez,Atacante,Nacional Potosí,5


In [2080]:
assist_atacantes = (
    assist_atacantes
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "assistencias": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2081]:
assist_atacantes = assist_atacantes.sort_values("assistencias", ascending=False)

assist_atacantes = assist_atacantes.head(10)

assist_atacantes

,name,position,team,assistencias
422,William Álvarez,Atacante,Nacional Potosí,5
218,Julián Álvarez,Atacante,"Argentina, Atlético Madrid",4
408,Vangelis Pavlidis,Atacante,"Benfica, Grécia",4
253,Luis Gómez,Atacante,Once Caldas,4
282,Mateo Retegui,Atacante,"Al-Qadsiah, Itália",4
300,Mikel Oyarzabal,Atacante,"Espanha, Real Sociedad",3
313,Musa Barrow,Atacante,Al-Taawoun,3
142,Giacomo Raspadori,Atacante,"Atlético Madrid, Itália",3
413,Vinícius Júnior,Atacante,Real Madrid,3
288,Matías Ramírez,Atacante,Águilas Doradas,3


##### Assistências Geral

In [2082]:
assistencias = assistencias.merge(df, how="inner", on="player_id")

assistencias = assistencias[["name_x", "position_x", "team", "assistencias"]]

assistencias.head(3)

,name_x,position_x,team,assistencias
0,Evander,Meio-campo,Cincinnati,5
1,Evander,Meio-campo,Cincinnati,5
2,Evander,Meio-campo,Cincinnati,5


In [2083]:
assistencias = (
    assistencias
    .groupby(["name_x", "position_x"], as_index=False)
    .agg({
        "team": lambda x: ", ".join(sorted(x.unique())),
        "assistencias": "max"
    })
    .rename(columns={
        "name_x": "name",
        "position_x": "position"
    })
)

In [2084]:
assistencias = assistencias.sort_values("assistencias", ascending=False)

assistencias = assistencias.head(10)

assistencias

,name,position,team,assistencias
461,Evander,Meio-campo,Cincinnati,5
1685,William Álvarez,Atacante,Nacional Potosí,5
292,Cristian Barco,Meio-campo,Guabirá,5
917,Kingsley Coman,Meio-campo,"Al-Nassr, França",5
89,Alexis Saelemaekers,Meio-campo,"Bélgica, Milan",5
867,Julián Álvarez,Atacante,"Argentina, Atlético Madrid",4
676,Iván Colman,Meio-campo,Cusco,4
92,Alexis Vega,Meio-campo,"México, Toluca",4
794,Jordi Alba,Defensor,Inter Miami,4
155,Arda Güler,Meio-campo,"Real Madrid, Turquia",4
